In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import MeanShift

In [2]:
data = pd.read_table('checkins.dat', sep='|', skipinitialspace = True, header=[0], skiprows=[1], )
data.dropna(axis='rows', inplace=True)

/home/dmitrytrofimov/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 396634 entries, 1 to 1021964
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                396634 non-null  object 
 1   user_id           396634 non-null  float64
 2   venue_id          396634 non-null  float64
 3   latitude          396634 non-null  float64
 4   longitude         396634 non-null  float64
 5   created_at        396634 non-null  object 
dtypes: float64(4), object(2)
memory usage: 21.2+ MB


In [6]:
data.head()

,id,user_id,venue_id,latitude,longitude,created_at
1,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
3,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
7,984291,105054.0,5222.0,45.523452,-122.676207,2012-04-21 17:39:22
9,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46
10,984232,93870.0,380645.0,33.448377,-112.074037,2012-04-21 17:38:18


In [7]:
X = data.iloc[:100000]

In [14]:
X[['latitude', 'longitude']]

,latitude,longitude
1,38.895112,-77.036366
3,33.800745,-84.410520
7,45.523452,-122.676207
9,40.764462,-111.904565
10,33.448377,-112.074037
...,...,...
233788,33.575000,-117.725556
233789,37.629349,-122.400087
233793,29.762884,-95.383061
233797,32.802955,-96.769923


In [12]:
X.rename(columns={'id    ': 'id', 'user_id ':'user_id', 'venue_id ':'venue_id',\
                  'latitude      ':'latitude', 'longitude     ':'longitude','created_at      ':'created_at'}, inplace=True)

/home/dmitrytrofimov/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [15]:
bandwidth = 0.1
clustering = MeanShift(bandwidth=bandwidth).fit(X[['latitude', 'longitude']])

In [22]:
unique, counts = np.unique(clustering.labels_, return_counts=True)

In [23]:
labels_counts = dict(zip(unique, counts))

In [25]:
labels_counts_filtered = {k:v for k,v in labels_counts.items() if v > 15}

In [26]:
labels_counts_filtered

{0: 12506,
 1: 4692,
 2: 3994,
 3: 3363,
 4: 3526,
 5: 2409,
 6: 2297,
 7: 1601,
 8: 1526,
 9: 1378,
 10: 1298,
 11: 1081,
 12: 1006,
 13: 1007,
 14: 714,
 15: 868,
 16: 870,
 17: 645,
 18: 808,
 19: 807,
 20: 612,
 21: 722,
 22: 754,
 23: 747,
 24: 539,
 25: 656,
 26: 580,
 27: 577,
 28: 679,
 29: 564,
 30: 594,
 31: 907,
 32: 449,
 33: 502,
 34: 452,
 35: 104,
 36: 431,
 37: 410,
 38: 388,
 39: 400,
 40: 369,
 41: 367,
 42: 384,
 43: 347,
 44: 345,
 45: 342,
 46: 314,
 47: 273,
 48: 314,
 49: 316,
 50: 355,
 51: 281,
 52: 336,
 53: 271,
 54: 293,
 55: 246,
 56: 263,
 57: 258,
 58: 254,
 59: 243,
 60: 229,
 61: 291,
 62: 182,
 63: 155,
 64: 137,
 65: 186,
 66: 193,
 67: 197,
 68: 196,
 69: 191,
 70: 189,
 71: 187,
 72: 203,
 73: 178,
 74: 192,
 75: 169,
 76: 173,
 77: 153,
 78: 157,
 79: 220,
 80: 199,
 81: 164,
 82: 162,
 83: 126,
 84: 155,
 85: 152,
 86: 56,
 87: 100,
 88: 141,
 89: 142,
 90: 190,
 91: 117,
 92: 138,
 93: 135,
 94: 133,
 95: 131,
 96: 134,
 97: 122,
 98: 130,
 99: 1

In [27]:
X['cluster'] = clustering.labels_

<ipython-input-27-a925b5bed6f9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['cluster'] = clustering.labels_


In [31]:
clusters_labels = labels_counts_filtered.keys()

In [36]:
centers = clustering.cluster_centers_

In [37]:
X['center_long'] = X.apply(lambda row: centers[row.cluster][0], axis=1)
X['center_lat'] = X.apply(lambda row: centers[row.cluster][1], axis=1)

<ipython-input-37-5a27d8e8b042>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['center_long'] = X.apply(lambda row: centers[row.cluster][0], axis=1)
<ipython-input-37-5a27d8e8b042>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['center_lat'] = X.apply(lambda row: centers[row.cluster][1], axis=1)


In [40]:
X

,id,user_id,venue_id,latitude,longitude,created_at,cluster,center_long,center_lat
1,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47,5,38.886165,-77.048783
3,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43,7,33.766636,-84.393289
7,984291,105054.0,5222.0,45.523452,-122.676207,2012-04-21 17:39:22,30,45.523483,-122.676280
9,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46,65,40.759600,-111.896078
10,984232,93870.0,380645.0,33.448377,-112.074037,2012-04-21 17:38:18,1,33.449438,-112.002140
...,...,...,...,...,...,...,...,...,...
233788,123888,1019078.0,1105563.0,33.575000,-117.725556,2011-12-11 05:38:19,50,33.650896,-117.752074
233789,124366,771129.0,329847.0,37.629349,-122.400087,2011-12-11 04:55:34,4,37.688682,-122.409330
233793,125556,804990.0,254046.0,29.762884,-95.383061,2011-12-11 06:54:20,25,29.762698,-95.382314
233797,127250,526214.0,34473.0,32.802955,-96.769923,2011-12-11 10:05:38,19,32.803021,-96.769897


In [41]:
offices = np.array([[33.751277, -118.188740],

[25.867736, -80.324116],

[51.503016, -0.075479],

[52.378894, 4.885084],

[39.366487, 117.036146],

[-33.868457, 151.205134]])

In [45]:
from scipy.spatial.distance import euclidean

In [47]:
def find_20_nearest(offices, clusters, cluster_centers):
    dict_cluster_mindist = {}
    for cluster in clusters:
        distances_to_offices = []
        for office in offices:
            distances_to_offices.append(euclidean(cluster_centers[cluster], office))
        dict_cluster_mindist[cluster] = min(distances_to_offices)
    return dict(sorted(dict_cluster_mindist.items(), key=lambda item: item[1]))

In [48]:
res_dict = find_20_nearest(offices, clusters_labels, centers)

In [56]:
nearest_20 = list(res_dict.items())[:20]

In [57]:
nearest_20

[(420, 0.007834758163107856),
 (370, 0.009353316185992226),
 (419, 0.022674066158385495),
 (58, 0.05005829482278787),
 (51, 0.07084773242719973),
 (29, 0.13410903336184654),
 (167, 0.16740596425035326),
 (92, 0.18887596060185083),
 (87, 0.19577945647763628),
 (42, 0.21181053682436798),
 (291, 0.2222332907317907),
 (320, 0.2713007595066735),
 (119, 0.2949788868004569),
 (55, 0.3022701186924605),
 (27, 0.3047305030784069),
 (11, 0.3148837903362732),
 (32, 0.3388104702511318),
 (159, 0.3408456533220572),
 (17, 0.37868750125029754),
 (47, 0.3867062248427277)]

In [61]:
answer = centers[420]

In [62]:
def write_ans(answer):
    with open('task1.txt', 'w') as task1:
        task1.write(' '.join(map(str, answer)))

In [63]:
write_ans(answer)